In [3]:
from ase.units import kB, mol, kcal, kJ, Hartree, Bohr

In [9]:
Bohr

0.5291772105638411

In [2]:
import nbformat

def generate_top_level_toc(notebook_path):
    with open(notebook_path, 'r', encoding='utf-8') as f:
        nb = nbformat.read(f, as_version=4)

    toc = "## Table of Contents\n\n"
    for cell in nb.cells:
        if cell.cell_type == 'markdown':
            lines = cell.source.split('\n')
            for line in lines:
                if line.startswith('# ') and not line.startswith('##') and not line.startswith('###'):
                    header = line.strip('# ').strip()
                    link = header.lower().replace(' ', '-')
                    toc += f"{len(toc.splitlines())}. [{header}](#{link})\n"

    return toc

# Replace 'your_notebook.ipynb' with the path to your notebook
toc_markdown = generate_top_level_toc('analyse.ipynb')
print(toc_markdown)

## Table of Contents

2. [Research data supporting "Reliable Insights into Surface Chemistry Enabled by Combining Experiments with Fast and Accurate Theory"](#research-data-supporting-"reliable-insights-into-surface-chemistry-enabled-by-combining-experiments-with-fast-and-accurate-theory")
3. [*H*<sub>ads</sub> from the experimental literature](#*h*<sub>ads</sub>-from-the-experimental-literature)
4. [Computing *E*<sub>rlx</sub>, *E*<sub>therm</sub> and *E*<sub>ZPV</sub> from DFT](#computing-*e*<sub>rlx</sub>,-*e*<sub>therm</sub>-and-*e*<sub>zpv</sub>-from-dft)
5. [Computing *E*<sub>int</sub> with the SKZCAM protocol](#computing-*e*<sub>int</sub>-with-the-skzcam-protocol)
6. [Contributions for the cohesive and conformational energy from WFT in selected systems](#contributions-for-the-cohesive-and-conformational-energy-from-wft-in-selected-systems)
7. [Contribution for Dissociation of CH₃OH and H₂O Clusters on MgO(001)](#contribution-for-dissociation-of-ch₃oh-and-h₂o-clusters-on-mgo(001)

In [ ]:
co2_xc_skzcam_cluster_size_dict = {functional: {} for functional in crystal_xc_func_ensemble['MgO']}
co2_xc_skzcam_cluster_eint_dict = {functional: {} for functional in crystal_xc_func_ensemble['MgO']}

for functional in crystal_xc_func_ensemble['MgO']:
    molecule = 'CO2 Chemisorbed'
    molecule_label = molecule.replace(' ', '_')
    num_monomer = 1
    for cluster_number in ['1','2','3','4','5','6','7']:
        filedir = f'Data/Miscellaneous/Error_Validation_CO2_MgO/{functional}/Cluster_{cluster_number}'
        outputname = f'MP2_awCV'
        try:
            # First get DZ Eint value
            mp2_total_eint = calculate_skzcam_eint(filedir = filedir, outputname = outputname, code='orca', method='mp2_total', basis='DZ')/num_monomer
            # If dictionary does not exist, create it
            if cluster_number not in co2_xc_skzcam_cluster_eint_dict[functional]:
                co2_xc_skzcam_cluster_eint_dict[functional][cluster_number] = {}
                co2_xc_skzcam_cluster_size_dict[functional][cluster_number] = {}
            co2_xc_skzcam_cluster_eint_dict[functional][cluster_number][f'MP2 awCVDZ'] = mp2_total_eint
            co2_xc_skzcam_cluster_size_dict[functional][cluster_number] = get_skzcam_cluster_size(f'{filedir}/Surface/{outputname}DZ.orca.out')
        except:
            pass

        try:
            # Then get CBS(DZ/TZ) Eint value
            hf_scf_eint = calculate_skzcam_eint(filedir = filedir, outputname = outputname, code='orca', method='mp2_hf', basis=['DZ','TZ'], cbs_type='scf_energy')/num_monomer
            mp2_corr_eint = calculate_skzcam_eint(filedir = filedir, outputname = outputname, code='orca', method='mp2_corr', basis=['DZ','TZ'], cbs_type='correlation_energy')/num_monomer
            co2_xc_skzcam_cluster_eint_dict[functional][cluster_number][f'MP2 CBS(awCVDZ/awCVTZ)'] = hf_scf_eint + mp2_corr_eint
        except:
            pass

        try:
            # Then get CBS(TZ/QZ) Eint value
            hf_scf_eint = calculate_skzcam_eint(filedir = filedir, outputname = outputname, code='orca', method='mp2_hf', basis=['TZ','QZ'], cbs_type='scf_energy')/num_monomer
            mp2_corr_eint = calculate_skzcam_eint(filedir = filedir, outputname = outputname, code='orca', method='mp2_corr', basis=['TZ','QZ'], cbs_type='correlation_energy')/num_monomer
            co2_xc_skzcam_cluster_eint_dict[functional][cluster_number][f'MP2 CBS(awCVTZ/awCVQZ)'] = hf_scf_eint + mp2_corr_eint
        except:
            pass
        
        outputname = f'LNOCCSDT_awCV'
        try:
            # Get the CBS(DZ/TZ) LNO-CCSD(T) Eint value
            hf_scf_eint = calculate_skzcam_eint(filedir = filedir, outputname = outputname, code='mrcc', method='hf', basis=['DZ','TZ'], cbs_type='scf_energy')/num_monomer
            ccsdt_corr_eint = calculate_skzcam_eint(filedir = filedir, outputname = outputname, code='mrcc', method='lccsdt_corr', basis=['DZ','TZ'], cbs_type='correlation_energy')/num_monomer
            ccsd_corr_eint = calculate_skzcam_eint(filedir = filedir, outputname = outputname, code='mrcc', method='lccsd_corr', basis=['DZ','TZ'], cbs_type='correlation_energy')/num_monomer
            mp2_corr_eint = calculate_skzcam_eint(filedir = filedir, outputname = outputname, code='mrcc', method='lmp2_corr', basis=['DZ','TZ'], cbs_type='correlation_energy')/num_monomer
            co2_xc_skzcam_cluster_eint_dict[functional][cluster_number][f'LNO-CCSD(T) CBS({basis_type}DZ/{basis_type}TZ)'] = hf_scf_eint + ccsdt_corr_eint
            co2_xc_skzcam_cluster_eint_dict[functional][cluster_number][f'LNO-CCSD CBS({basis_type}DZ/{basis_type}TZ)'] = hf_scf_eint + ccsd_corr_eint
            co2_xc_skzcam_cluster_eint_dict[functional][cluster_number][f'LMP2 CBS({basis_type}DZ/{basis_type}TZ)'] = hf_scf_eint + mp2_corr_eint
        except:
            pass

In [1]:
import numpy as np
from ase import units
from scipy.stats import linregress
from datetime import datetime
from ase.units import mol, kcal, kJ, Hartree, Bohr
import pandas as pd
from ase import io
import re


# Some basic conversion factors
cm1_to_eV = 1 / 8065.54429
hundredcm1 = 100 * cm1_to_eV * 1000
kcalmol_to_meV = kcal / mol * 1000
kjmol_to_meV = kJ / mol * 1000
mha_to_meV = Hartree

In [2]:
def get_mrcc_walltime(filename):
    """
    Reads the walltime from the mrcc.out file.
        
    Parameters
    ----------
    filename : str
        The location of the 'mrcc.out' file to read from.
   
   Returns
    -------
    float
        The walltime in seconds.
        
    Notes
    -----
    This function reads the 'mrcc.out' file and extracts the start and end time information to calculate the total walltime taken by the MRCC calculation. The function calculates the time duration between the timestamps found in the file and returns it in seconds.
    """

    # Get the start and end time from the file
    orig_time = datetime(1, 1, 1, 0, 0)
    total_time = datetime(1, 1, 1, 0, 0)

    f = open(filename)
    a = f.readlines()
    b1 = datetime.strptime(
        a[19].split()[1] + "-" + a[19].split()[2], "%Y-%m-%d-%H:%M:%S"
    )
    b2 = datetime.strptime(
        a[-3].split()[1] + "-" + a[-3].split()[2], "%Y-%m-%d-%H:%M:%S"
    )
    total_time = total_time + (b2 - b1)
    f.close()
    return (total_time - orig_time).total_seconds()

In [7]:
def get_orca_walltime(filename):
    """
    Reads the walltime from the mrcc.out file.
        
    Parameters
    ----------
    filename : str
        The location of the 'mrcc.out' file to read from.
   
   Returns
    -------
    float
        The walltime in seconds.
        
    Notes
    -----
    This function reads the 'orca.out' file and extracts the start and end time information to calculate the total walltime in seconds taken by the ORCA calculation. The function calculates the time duration between the timestamps found in the file and returns it in seconds.
    """

    # Get the start and end time from the file
    orig_time = datetime(1, 1, 1, 0, 0)
    total_time = datetime(1, 1, 1, 0, 0)

    f = open(filename)
    line = f.readlines()[-1].split()
    f.close()

    total_time_minutes = float(line[3])*24*60*60+ float(line[5])*60*60 + float(line[7])*60 + float(line[9])
    return total_time_minutes

In [12]:
get_orca_walltime('Data/05b-Eint_SKZCAM/r-TiO2/H2O/Cluster_7/Molecule-Surface/MP2_aVDZ.orca.out')

166033.0

In [13]:
1*24*60*60+22*60*60+7*60+13

166033

In [11]:
2*60*60+47*60+28

10048

In [3]:
line = 'TOTAL RUN TIME: 0 days 0 hours 0 minutes 12 seconds 900 msec'

In [5]:
float(line.split()[3])*24*60+ float(line.split()[5])*60 + float(line.split()[7]) + float(line.split()[9])

['TOTAL',
 'RUN',
 'TIME:',
 '0',
 'days',
 '0',
 'hours',
 '0',
 'minutes',
 '12',
 'seconds',
 '900',
 'msec']

In [ ]:
fig, axs = plt.subplots(figsize=(3.365,3.5),dpi=600, sharey=True, constrained_layout=True) #,gridspec_kw={'width_ratios':[0.65,2]})

autoskzcam_n2o_parallel_hads = autoskzcam_final_hads_dict['MgO']['N2O Parallel']['Final'][0]
autoskzcam_n2o_parallel_hads_error = autoskzcam_final_hads_dict['MgO']['N2O Parallel']['Final'][1]
dft_xc_n2o_parallel_hads = [dft_eads_true_dict['MgO']['N2O Parallel'][xc_func] + autoskzcam_final_hads_dict['MgO']['N2O Parallel']['DFT DeltaH'][0] for xc_func in crystal_xc_func_ensemble['MgO']]
dft_xc_n2o_parallel_hads_error = [autoskzcam_final_hads_dict['MgO']['N2O Parallel']['DFT DeltaH'][1] for xc_func in crystal_xc_func_ensemble['MgO']]
autoskzcam_n2o_tilted_hads = autoskzcam_final_hads_dict['MgO']['N2O Tilted']['Final'][0]
autoskzcam_n2o_tilted_hads_error = autoskzcam_final_hads_dict['MgO']['N2O Tilted']['Final'][1]
dft_xc_n2o_tilted_hads = [dft_eads_true_dict['MgO']['N2O Tilted'][xc_func] + autoskzcam_final_hads_dict['MgO']['N2O Tilted']['DFT DeltaH'][0] for xc_func in crystal_xc_func_ensemble['MgO']]
dft_xc_n2o_tilted_hads_error = [autoskzcam_final_hads_dict['MgO']['N2O Tilted']['DFT DeltaH'][1] for xc_func in crystal_xc_func_ensemble['MgO']]

rutile_n2o_expt_hads = float(expt_hads_df.loc[(expt_hads_df['Surface'] == 'MgO') & (expt_hads_df['Molecule'] == 'N2O'), 'Hads'].values[0])
rutile_n2o_expt_error = float(expt_hads_df.loc[(expt_hads_df['Surface'] == 'MgO') & (expt_hads_df['Molecule'] == 'N2O'), 'Error'].values[0])

axs.errorbar(list(range(7)), [autoskzcam_n2o_parallel_hads] + dft_xc_n2o_parallel_hads, yerr = [autoskzcam_n2o_parallel_hads_error] + dft_xc_n2o_parallel_hads_error, color=color_dict['red'], alpha=0.7, ls='--', marker='o', capsize=2, label='Parallel')
axs.errorbar(list(range(7)), [autoskzcam_n2o_tilted_hads] + dft_xc_n2o_tilted_hads, yerr = [autoskzcam_n2o_tilted_hads_error] + dft_xc_n2o_tilted_hads_error, color=color_dict['blue'], alpha=0.7, ls='--', marker='o', capsize=2, label='Tilted')

axs.plot([-1,9], [rutile_n2o_expt_hads, rutile_n2o_expt_hads], color='k', linestyle='--', label=r'Expt. $H_\textrm{ads}$')
axs.fill_between([-1,9], rutile_n2o_expt_hads - rutile_n2o_expt_error, rutile_n2o_expt_hads + rutile_n2o_expt_error, color=color_dict['black'],alpha=0.1, edgecolor='none')


axs.set_xticks(list(range(7)))
axs.set_xticklabels([r'\textbf{autoSKZCAM}'] + [convert_to_nice_labels['xc_functionals'][xc_func] for xc_func in crystal_xc_func_ensemble['MgO']],rotation=90,ha='center')
axs.legend(frameon=False, fontsize=8)
axs.set_ylabel(r'$H_\textrm{ads}$ [meV]')
axs.set_ylim([-600,-300])
axs.set_xlim([-0.5,6.5])

plt.savefig('Figures/SI_Figure-Hads_rutile_N2O_Configurations.png')